In [55]:
#Imports
import pandas as pd
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
import urllib
import numpy as np

In [56]:
# Watermark
print('Nathan Young\nJunior Data Analyst\nCenter for the Study of Free Enterprise')
%load_ext watermark
%watermark -a "Western Carolina University" -u -d -p pandas

Nathan Young
Junior Data Analyst
Center for the Study of Free Enterprise
The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Western Carolina University 
last updated: 2020-02-22 

pandas 1.0.1


In [57]:
# Create backups
df_backup = pd.read_csv('./Updates/STG_FRED_Resident_Population_by_County_Thousands_of_Persons.txt')
df_backup.to_csv('./Backups/STG_FRED_Resident_Population_by_County_Thoudands_of_Persons_BACKUP.txt')

In [58]:
# Getting and reading new data 
df = pd.read_excel("https://geofred.stlouisfed.org/api/download.php?theme=pubugn&colorCount=5&reverseColors=false&intervalMethod=fractile&displayStateOutline=true&lng=-89.96&lat=40.78&zoom=4&showLabels=true&showValues=true&regionType=county&seriesTypeId=1549&attributes=Not+Seasonally+Adjusted%2C+Annual%2C+Thousands+of+Persons&aggregationFrequency=Annual&aggregationType=Average&transformation=lin&date=2030-01-01&type=xls&startDate=1970-01-01&endDate=2030-01-01&mapWidth=999&mapHeight=582&hideLegend=false", skiprows=1)
df.head(2)

,Series ID,Region Name,Region Code,1970,1971,1972,1973,1974,1975,1976,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,SCABBE1POP,"Abbeville County, SC",45001,21.112,21.5,21.5,21.6,21.9,22.1,22.2,...,25.053,25.310,25.060,25.000,24.880,24.775,24.775,24.633,24.560,24.541
1,LAACAD0POP,"Acadia Parish, LA",22001,52.109,52.5,52.6,52.8,52.7,53.1,53.8,...,60.115,61.875,61.853,61.991,62.294,62.664,62.681,62.793,62.514,62.190


In [59]:
# Filter data to display only North Carolina
filter1 = df['Region Name'].str.contains(', NC')
df_nc = df[filter1]
df_nc.head(2)

,Series ID,Region Name,Region Code,1970,1971,1972,1973,1974,1975,1976,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
24,NCALAM1POP,"Alamance County, NC",37001,96.502,98.5,99.7,99.9,99.3,98.2,98.7,...,150.244,151.440,152.696,153.282,154.199,155.425,157.10,160.392,163.529,166.436
34,NCALEX3POP,"Alexander County, NC",37003,19.466,20.3,21.2,22.1,22.7,22.9,23.1,...,36.822,37.235,37.028,36.925,37.016,36.992,36.96,37.142,37.146,37.353


In [60]:
# Set index to Series ID
df_nc.set_index(df_nc['Series ID'], inplace = True)
df_nc.head(2)

,Series ID,Region Name,Region Code,1970,1971,1972,1973,1974,1975,1976,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Series ID,,,,,,,,,,,,,,,,,,,,,
NCALAM1POP,NCALAM1POP,"Alamance County, NC",37001,96.502,98.5,99.7,99.9,99.3,98.2,98.7,...,150.244,151.440,152.696,153.282,154.199,155.425,157.10,160.392,163.529,166.436
NCALEX3POP,NCALEX3POP,"Alexander County, NC",37003,19.466,20.3,21.2,22.1,22.7,22.9,23.1,...,36.822,37.235,37.028,36.925,37.016,36.992,36.96,37.142,37.146,37.353


In [61]:
# Drop Series ID column
df_nc.drop('Series ID', axis = 1, inplace = True)
df_nc.head(2)

C:\Users\natha\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Region Name,Region Code,1970,1971,1972,1973,1974,1975,1976,1977,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Series ID,,,,,,,,,,,,,,,,,,,,,
NCALAM1POP,"Alamance County, NC",37001,96.502,98.5,99.7,99.9,99.3,98.2,98.7,98.5,...,150.244,151.440,152.696,153.282,154.199,155.425,157.10,160.392,163.529,166.436
NCALEX3POP,"Alexander County, NC",37003,19.466,20.3,21.2,22.1,22.7,22.9,23.1,23.6,...,36.822,37.235,37.028,36.925,37.016,36.992,36.96,37.142,37.146,37.353


In [62]:
# Save file to tab delimited txt for upload to SSMS
df_nc.to_csv('./Updates/STG_FRED_Resident_Population_by_County_Thousands_of_Persons.txt', sep = '\t')

In [63]:
#Reset Index for upload to database
df_nc = df_nc.reset_index()    

In [64]:
column_list = df_nc.columns.values
for i in column_list:
    df_nc.loc[df_nc[i].isnull(),i]=0

In [65]:
#Connect to database and create cursor
con = pyodbc.connect('Driver={SQL Server};'
                      'Server=TITANIUM-BOOK;'
                      'Database=DataDashboard;'
                      'Trusted_Connection=yes;',
                    autocommit=True)

c = con.cursor()

In [66]:
#Drop old backup table
c.execute('drop table STG_FRED_Resident_Population_by_County_Thousands_of_Persons_BACKUP')

In [67]:
#Create new backup
c.execute('''sp_rename 'dbo.STG_FRED_Resident_Population_by_County_Thousands_of_Persons','STG_FRED_Resident_Population_by_County_Thousands_of_Persons_BACKUP';''')

In [68]:
c.execute('''USE [DataDashboard]

SET ANSI_NULLS ON


SET QUOTED_IDENTIFIER ON

CREATE TABLE [dbo].[STG_FRED_Resident_Population_by_County_Thousands_of_Persons](
	[Series ID] [varchar](14) NULL,
	[Region Name] [varchar](23) NULL,
	[Region Code] [int] NULL,
	[1975] [float] NULL,
	[1976] [float] NULL,
	[1977] [float] NULL,
	[1978] [float] NULL,
	[1979] [float] NULL,
	[1980] [float] NULL,
	[1981] [float] NULL,
	[1982] [float] NULL,
	[1983] [float] NULL,
	[1984] [float] NULL,
	[1985] [float] NULL,
	[1986] [float] NULL,
	[1987] [float] NULL,
	[1988] [float] NULL,
	[1989] [float] NULL,
	[1990] [float] NULL,
	[1991] [float] NULL,
	[1992] [float] NULL,
	[1993] [float] NULL,
	[1994] [float] NULL,
	[1995] [float] NULL,
	[1996] [float] NULL,
	[1997] [float] NULL,
	[1998] [float] NULL,
	[1999] [float] NULL,
	[2000] [float] NULL,
	[2001] [float] NULL,
	[2002] [float] NULL,
	[2003] [float] NULL,
	[2004] [float] NULL,
	[2005] [float] NULL,
	[2006] [float] NULL,
	[2007] [float] NULL,
	[2008] [float] NULL,
	[2009] [float] NULL,
	[2010] [float] NULL,
	[2011] [float] NULL,
	[2012] [float] NULL,
	[2013] [float] NULL,
	[2014] [float] NULL,
	[2015] [float] NULL,
	[2016] [float] NULL,
	[2017] [float] NULL,
	[2018] [float] NULL,
    [2019] [float] NULL,
    [2020] [float] NULL,
    [2021] [float] NULL,
    [2022] [float] NULL,
    [2023] [float] NULL,
    [2024] [float] NULL,
    [2025] [float] NULL,
    [2026] [float] NULL,
    [2027] [float] NULL,
    [2028] [float] NULL,
    [2029] [float] NULL,
    [2030] [float] NULL
) ON [PRIMARY]''')

In [69]:
params = urllib.parse.quote_plus(r'Driver={SQL Server};' 
                                 r'Server=TITANIUM-BOOK;'
                                 r'Database=DataDashboard;'
                                 r'Trusted_Connection=yes;')

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

#warning: discard old table if exists
df_nc.to_sql('STG_FRED_Resident_Population_by_County_Thousands_of_Persons', con=engine, if_exists='replace', index=False)